In [34]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
from py2neo import Graph
import faiss
import numpy as np

# Kết nối với Neo4j và lấy dữ liệu môn học từ ontology
neo4j_graph = Graph("neo4j+s://503c5eaf.databases.neo4j.io", auth=("neo4j", "OplNn6X6ehDa3GhvqJECbHcfXSe_L9juCTK2SGuvDaQ"), name="neo4j")

# Truy vấn để lấy thông tin các môn học
query = """
  MATCH (ancestor:Resource {uri: 'http://localhost/ontologies/2024/10/11/edu_program#monHoc'})
  MATCH (n:Resource)-[:rdfs__subClassOf*]->(ancestor)
  MATCH (instance:Resource)-[:rdf__type]->(n)
  RETURN DISTINCT instance.ns0__maMonHoc AS code, instance.rdfs__label AS label, id(instance) AS elementId
"""
results = neo4j_graph.run(query).data()

# Khởi tạo PhoBERT để mã hóa các nhãn môn học
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
model = AutoModel.from_pretrained("vinai/phobert-base").to(device)

# Tạo vector biểu diễn ngữ nghĩa cho các môn học
labels = [result['label'] for result in results]
element_ids = [result['elementId'] for result in results]
encoded_labels = []

for label in labels:
    inputs = tokenizer(label, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
        encoded_labels.append(embeddings)

encoded_labels = np.vstack(encoded_labels)

# Lập chỉ mục và tìm kiếm với FAISS
dimension = encoded_labels.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(encoded_labels)

# Hàm tìm kiếm ngữ nghĩa
def semantic_search(query, top_k=5):
    # Mã hóa câu truy vấn
    inputs = tokenizer(query, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        query_vector = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
    
    # Tìm kiếm trong FAISS
    distances, indices = index.search(query_vector, top_k)
    
    # Trả về kết quả tìm kiếm
    results_list = []
    for i, idx in enumerate(indices[0]):
        result = {
            "label": labels[idx],
            "elementId": element_ids[idx],
            "similarity": 1 / (1 + distances[0][i])
        }
        results_list.append(result)
    return results_list

# Ví dụ tìm kiếm
query_text = "môn học về an toàn thông tin"
search_results = semantic_search(query_text)

print("Kết quả tìm kiếm:")
for result in search_results:
    print(f"Label: {result['label']}, ElementId: {result['elementId']}, Similarity: {result['similarity']:.4f}")


Kết quả tìm kiếm:
label: Thương mại điện tử, elementId: 64, similarity: 0.4914
label: Quản trị mạng, elementId: 214, similarity: 0.4574
label: Phương pháp tính, elementId: 96, similarity: 0.4446
label: Giao tiếp người máy, elementId: 11, similarity: 0.4414
label: Cấu trúc máy tính và vi xử lý, elementId: 84, similarity: 0.4398
